# **Statistical Analyst Time Series Using Sunspot Data**
---
pada tahap ini kita akan melakukan preprocesing data lebih lanjut dengan menggunakan statistical analyst 

## **1.Statistical Analyst Using Sunspot Data**

### **1.1 Import Library**
---
pertama kita akan melakukan import library yang dibutuhkan untuk proses analisis statistik pada data sunspot

In [ ]:
# import library yang dibutuhkan 
import numpy as np                              # import library untuk komputasi numerik 
import pandas as pd                             # import library untuk pengolahan datafrane 
import matplotlib.pyplot as plt                 # import library untuk visualisasi data dengan matplotlib 
import seaborn as sns                           # import library untuk visualisasi data dengan seaborn 

from warnings import simplefilter               # import library untuk handling error 
from scipy import stats
from statsmodels.tsa.arima_model import ARIMA   # import library untuk pengolahan statistik dengan model ARIMA (Autoregresive moving average)
from statsmodels.graphics.api import qqplot     # import library untuk pengolahan statistik dalam bentuk visualisasi data

# definisikan variabel untuk memfilter error yang ada 
simplefilter('ignore')